# Linear Regression

Welcome to your first assignment. This exercise gives you a brief introduction to linear regression. The exercise is to be implemented in Python. Even if you've used Python before, this will help familiarize you with functions we'll need.  

**Instructions:**
- You will be using Python 3.
- Avoid using for-loops and while-loops, unless you are explicitly told to do so.
- Do not modify the (# GRADED FUNCTION [function name]) comment in some cells. Your work would not be graded if you change this. Each cell containing that comment should only contain one function.
- After coding your function, run the cell right below it to check if your result is correct.

**After this assignment you will:**
- Be able to implement linear regression model using statsmodels, scikit-learn, and tensorflow
- Work with simulated non-linear dataset
- Compare model performance (quality of fit) of both models

Let's get started!

## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. You only need to write code between the ### START CODE HERE ### and ### END CODE HERE ### comments. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run Cell" (denoted by a play symbol) in the upper bar of the notebook. 

We will often specify "(≈ X lines of code)" in the comments to tell you about how much code you need to write. It is just a rough estimate, so don't feel bad if your code is longer or shorter.

In [1]:
import os
import numpy as np

try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except: pass

import pandas as pd

import tensorflow as tf
from tensorflow.python.layers import core as core_layers
try:
    from mpl_toolkits.mplot3d import Axes3D
except: pass

In [2]:
def reset_graph(seed=42):
    """
    Utility function to reset current tensorflow computation graph
    and set the random seed 
    """
    # to make results reproducible across runs
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

## We use artificial data for the following two specifications of regression:

### Linear Regression

$ y(x) = a + b_1 \cdot X_1 + b_2 \cdot X_2 + b_3 \cdot X_3 + \sigma \cdot \varepsilon $ 

where $ \varepsilon \sim N(0, 1) $ is a Gaussian noise, and $ \sigma $ is its volatility, 
with the following choice of parameters:

$ a = 1.0 $

$ b_1, b_2, b_3 = (0.5, 0.2, 0.1) $

$ \sigma = 0.1 $

$ X_1, X_2, X_3 $ will be uniformally distributed in $ [-1,1] $

### Non-Linear Regression

$ y(x) = a + w_{00} \cdot X_1 + w_{01} \cdot X_2 + w_{02} \cdot X_3 + + w_{10} \cdot X_1^2 
+ w_{11} \cdot X_2^2 + w_{12} \cdot X_3^2 +  \sigma \cdot \varepsilon $ 

where

$ w = [[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]]  $

and the rest of parameters is as above, with the same values of $ X_i $

### Generate Data

In [3]:
def generate_data(n_points=10000, n_features=3, use_nonlinear=True, 
                    noise_std=0.1, train_test_split = 4):
    """
    Arguments:
    n_points - number of data points to generate
    n_features - a positive integer - number of features
    use_nonlinear - if True, generate non-linear data
    train_test_split - an integer - what portion of data to use for testing
    
    Return:
    X_train, Y_train, X_test, Y_test, n_train, n_features
    """
    
    # Linear data or non-linear data?
    if use_nonlinear:
        weights = np.array([[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]])
    else:
        weights = np.array([1.0, 0.5, 0.2])
        
    bias = np.ones(n_points).reshape((-1,1))
    low = - np.ones((n_points,n_features),'float')
    high = np.ones((n_points,n_features),'float')
    
    X = np.random.uniform(low=low, high=high)
    noise = np.random.normal(size=(n_points, 1))
    noise_std = 0.1
    
    if use_nonlinear:
        Y = (weights[0,0] * bias + np.dot(X, weights[0, :]).reshape((-1,1)) + 
             np.dot(X*X, weights[1, :]).reshape([-1,1]) +
             noise_std * noise)
    else:
        Y = (weights[0] * bias + np.dot(X, weights[:]).reshape((-1,1)) + 
             noise_std * noise)
    
    n_test = int(n_points/train_test_split)
    n_train = n_points - n_test
    
    X_train = X[:n_train,:]
    Y_train = Y[:n_train].reshape((-1,1))

    X_test = X[n_train:,:]
    Y_test = Y[n_train:].reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, n_train, n_features

X_train, Y_train, X_test, Y_test, n_train, n_features = generate_data(use_nonlinear=False)
X_train.shape, Y_train.shape

((7500, 3), (7500, 1))

### Linear Regression with Numpy

In [4]:
# GRADED FUNCTION: numpy_lin_regress
def numpy_lin_regress(X_train, Y_train):
    """
    numpy_lin_regress - Implements linear regression model using numpy module
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    # number of features
    ndim = X_train.shape[1] 
    ### START CODE HERE ### (≈ 3 lines of code)
    
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    theta_numpy = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(Y_train)
    
    ### END CODE HERE ###
    return theta_numpy

In [5]:
### GRADED PART (DO NOT EDIT) ###
theta_numpy = numpy_lin_regress(X_train, Y_train)
theta_numpy.squeeze()
### GRADED PART (DO NOT EDIT) ###

array([ 1.00234467,  0.99731268,  0.50003552,  0.19935747])

### Linear Regression with Sklearn

In [6]:
# GRADED FUNCTION: sklearn_lin_regress
def sklearn_lin_regress(X_train, Y_train):
    """
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    ndim = X_train.shape[1] 
    ### START CODE HERE ### (≈ 3 lines of code)
    try:
        from sklearn.linear_model import LinearRegression
    except ImportError:
        raise("ImportError: No module named sklearn.linear_model found")
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    reg = LinearRegression(fit_intercept=False)
    reg.fit(X_train, Y_train)
    theta_sklearn = reg.coef_
    ### END CODE HERE ###
    return theta_sklearn

In [7]:
### GRADED PART (DO NOT EDIT) ###
theta_sklearn = sklearn_lin_regress(X_train, Y_train)
theta_sklearn.squeeze()
### GRADED PART (DO NOT EDIT) ###

array([ 1.00234467,  0.99731268,  0.50003552,  0.19935747])

### Linear Regression with Tensorflow

In [8]:
# GRADED FUNCTION: tf_lin_regress
def tf_lin_regress(X_train, Y_train):
    """
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    ndim = X_train.shape[1] 
    ### START CODE HERE ### (≈ 7-8 lines of code)
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    X = tf.constant(X_train, dtype=tf.float32, name="X")
    Y = tf.constant(Y_train, dtype=tf.float32, name="Y")
    XT = tf.transpose(X)
    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y)
    with tf.Session() as sess:
        theta_value = theta.eval()
    ### END CODE HERE ###
    return theta_value

In [9]:
### GRADED PART (DO NOT EDIT) ###
theta_tf = tf_lin_regress(X_train, Y_train)
theta_tf.squeeze()
### GRADED PART (DO NOT EDIT) ###

array([ 1.00234461,  0.99731261,  0.50003546,  0.19935751], dtype=float32)

In [10]:
# GRADED FUNCTION: run_normal_eq
def run_normal_eq(X_train, Y_train, X_test, Y_test, learning_rate=0.05):
    """
    Implements normal equation using tensorflow, trains the model using training data set
    Tests the model quality by computing mean square error (MSE) of the test data set
    
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    X_test  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_test - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    
    Return a tuple of:
        - np.array of size (k+1 by 1) of regression coefficients
        - mean square error (MSE) of the test data set
        - mean square error (MSE) of the training data set
    """
    ndim = X_train.shape[1]
    lr_mse_train = 0.
    lr_mse_test = 0.
    ### START CODE HERE ### (≈ 20 lines of code)
    X = tf.placeholder(tf.float32, [None, ndim], name="X")
    Y = tf.placeholder(tf.float32, [None, 1], name="Y")
    theta_in = tf.placeholder(tf.float32, [ndim + 1, None])
    data_plus_bias = tf.concat([tf.ones([tf.shape(X)[0], 1]), X], axis=1)
    XT = tf.transpose(data_plus_bias)
    
    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), Y)
    lr_mse = tf.reduce_mean(tf.square(tf.matmul(data_plus_bias, theta_in) - Y))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        theta_value = sess.run(theta, feed_dict={X: X_train, Y: Y_train})
        lr_mse_train = sess.run(lr_mse, feed_dict={X: X_train, Y: Y_train, theta_in: theta_value})
        lr_mse_test = sess.run(lr_mse, feed_dict={X: X_train, Y: Y_train, theta_in: theta_value})
    ### END CODE HERE ###
    return theta_value, lr_mse_train, lr_mse_test

### (DO NOT EDIT) ###
theta_value, lr_mse_train, lr_mse_test = run_normal_eq(X_train, Y_train, X_test, Y_test)
### (DO NOT EDIT) ###

In [11]:
### GRADED PART (DO NOT EDIT) ###
theta_value.squeeze()
### GRADED PART (DO NOT EDIT) ###

array([ 1.00234461,  0.99731261,  0.50003546,  0.19935751], dtype=float32)

In [12]:
# GRADED FUNCTION: run_mle
def run_mle(X_train, Y_train, X_test, Y_test, learning_rate=0.05, num_iter=5000):
    """
    Maximum likelihood Estimate (MLE)
    Tests the model quality by computing mean square error (MSE) of the test data set
    
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    X_test  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_test - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    
    Return a tuple of:
        - np.array of size (k+1 by 1) of regression coefficients
        - mean square error (MSE) of the test data set
        - mean square error (MSE) of the training data set
    """
    # create an instance of the Linear Regression model class  
    ndim = X_train.shape[1]
    loss = 0.
    std_model = 0.
    ### START CODE HERE ### (≈ 20 lines of code)
    X = tf.placeholder(tf.float32, [None, ndim], name="X")
    Y = tf.placeholder(tf.float32, [None, 1], name="Y")
    data_plus_bias = tf.concat([tf.ones([tf.shape(X)[0], 1]), X], axis=1)
    
    theta = tf.Variable(tf.random_normal([ndim + 2, 1]))
    output = tf.matmul(data_plus_bias, theta[:-1, :])
    gauss = tf.distributions.Normal(loc=0.0, scale=1.0)
    sigma = 0.0001 + tf.square(theta[-1])
    
    log_LL = tf.log(0.00001 + (1 / sigma) * gauss.prob((Y - output) / sigma))
    loss = -tf.reduce_mean(log_LL)
    train_step = (tf.train.AdamOptimizer(learning_rate).minimize(loss), loss)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(num_iter):
            (_, loss), weights = sess.run((train_step, theta), feed_dict={X: X_train, Y: Y_train})
        
        Y_test_predicted = sess.run(output, feed_dict={X: X_test})
        
        std_model = weights[-1] ** 2
    ### END CODE HERE ###
    return weights, loss, std_model

weights, loss, std_model = run_mle(X_train, Y_train, X_test, Y_test)

In [13]:
### GRADED PART (DO NOT EDIT) ###
weights.squeeze()
### GRADED PART (DO NOT EDIT) ###

array([ 1.00228989,  0.99725753,  0.50010562,  0.19931842, -0.31519783], dtype=float32)